In [1]:
from tkinter import *
from tkinter import ttk
from tkinter import messagebox
import pandas as pd
import cv2
import os
import json
from pytube import YouTube 
import ssl
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
import numpy as np
import PIL.Image, PIL.ImageTk
import threading
import math
import time
from tkinter.font import Font
from PyQt5 import QtWidgets
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel ,QPushButton, QWidget
from PyQt5.QtCore import pyqtSlot
import sys



#main_window = Tk()
#Frame styling (not working currently)
#main_window.wm_attributes('-type', 'splash')
#main_window.geometry("500x1000+0+0")
#main_window.configure(background= 'black')

#General Styling
#myFont = Font(family="IBM Plex Sans", weight="bold", size=108 )

# Frame update, can go soon
#count = 0
#count_str = StringVar()
#count_str.set('hey')

screen_width = 1920
screen_height = 1080

video_x_values = 0
video_y_values = 0


def get_video(filename):
    #clicked()
    class App:
        def __init__(self, window, window_title, video_source):
            self.window = window
            self.window.title(window_title)
            self.video_source = video_source

            # open video source
            self.vid = MyVideoCapture(video_source)

            # Create a canvas that can fit the above video source size
            self.canvas = Canvas(window, width = self.vid.width, height = self.vid.height)
            self.canvas.pack()

            global video_x_values
            global video_y_values

            print(video_x_values)
            print(video_y_values)
            
            # Move window down when screen width is filled
            if video_x_values + int(self.vid.width) > screen_width:
                video_y_values += (self.vid.height + 40)
                video_x_values = 0

            # Move Window based on previous window sizes
            self.window.geometry(f"+{video_x_values}+{video_y_values}")
            video_x_values += int(self.vid.width)

            # Frame Updates in ms
            self.delay = 150
            self.update()


            #self.window.mainloop()
        def update(self):
            ret, frame = self.vid.get_frame()
            if ret:
                self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(frame))
                self.canvas.create_image(0, 0, image = self.photo, anchor = NW)
            
            self.window.after(self.delay, self.update)

        

        

    class MyVideoCapture:
        def __init__(self, video_source):
            # Open the video source
            self.vid = cv2.VideoCapture(video_source)
            if not self.vid.isOpened():
                raise ValueError("Unable to open video source", video_source)
    
            # Get video source width and height
            self.original_width = self.vid.get(cv2.CAP_PROP_FRAME_WIDTH)
            self.original_height = self.vid.get(cv2.CAP_PROP_FRAME_HEIGHT)
            self.aspect_ratio = self.original_width / self.original_height
            
            # Desired Height of Video
            self.height = 120
            self.width = self.height * self.aspect_ratio

        def get_frame(self):
            if self.vid.isOpened():
                ret, frame = self.vid.read()
                if ret:
                    frame = cv2.resize(frame,(int(self.width), self.height))
                    # Return a boolean success flag and the current frame converted to BGR
                    return (ret, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                else:
                    return (ret, None)
            else:
                return (ret, None)

        # Release the video source when the object is destroyed
        def __del__(self):
            if self.vid.isOpened():
                self.vid.release()
            self.window.mainloop()




    # Remove Toplevel if want to show in main_window
    App(Toplevel(main_window), f"Video {filename}", filename)
    


def open_new_window():

    # Toplevel object which will
    # be treated as a new window
    #Edit this to change into adding videos to one window
    newWindow = Toplevel(main_window)
    newWindow.geometry("+0+0")
 
    # sets the title of the
    # Toplevel widget
    newWindow.title("search results")
 
    # sets the geometry of toplevel
    #newWindow.geometry("400x400")
 
    # A Label widget to show in toplevel
    Label(newWindow,
        text ="below is search json (hopefully)").pack()
 
    threading.Thread(target=final).start()

    Label(newWindow,
        textvariable = count_str).pack()



class App(QWidget):

    def __init__(self):
        super().__init__()
        self.title = 'PyQt5 button - pythonspot.com'
        self.left = 10
        self.top = 10
        self.width = 320
        self.height = 200
        self.initUI()
    
    def initUI(self):
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)
        
        button = QPushButton('PyQt5 button', self)
        button.setToolTip('This is an example button')
        button.move(100,70)
        button.clicked.connect(self.on_click)
        
        self.show()

    @pyqtSlot()
    def on_click(self):
        print('PyQt5 button click')

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = App()
    sys.exit(app.exec_())
  

PyQt5 button click
PyQt5 button click


SystemExit: 0

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
